<a href="https://colab.research.google.com/github/plus2net/numpy/blob/main/numpy_7_views_strides.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://www.plus2net.com/images/top2.jpg)        Read more on [Broadcasting  ](https://www.plus2net.com/python/numpy-broadcasting.php) | [ Numpy ](https://www.plus2net.com/python/numpy.php)

In [2]:
import numpy as np

a = np.arange(6)          # [0 1 2 3 4 5]
s = a[1:4]                # slice -> view
s[0] = 99
print(a)                  # [ 0 99  2  3  4  5]  changed!

c = a[1:4].copy()         # explicit copy
c[0] = -1
print(a[1:4])             # original unchanged now

[ 0 99  2  3  4  5]
[99  2  3]


In [3]:
M = np.arange(12, dtype=np.int32).reshape(3,4)
print(M.shape, M.dtype, M.strides)  # e.g. (3, 4), int32, (16, 4) on typical systems
# (16 bytes to move to next row, 4 bytes to move to next column)

row_view = M[::2, :]                 # step of 2 in rows
print(row_view.shape, row_view.strides)

col_view = M[:, ::2]                 # step of 2 in cols
print(col_view.shape, col_view.strides)

(3, 4) int32 (16, 4)
(2, 4) (32, 4)
(3, 2) (16, 8)


In [4]:
A = np.arange(12).reshape(3,4)         # default C-order
print(A.flags['C_CONTIGUOUS'], A.flags['F_CONTIGUOUS'])  # True, False

F = np.asfortranarray(A)
print(F.flags['C_CONTIGUOUS'], F.flags['F_CONTIGUOUS'])  # False, True

True False
False True


In [5]:
X = np.arange(9).reshape(3,3)
rv = X.ravel()     # likely a view
fl = X.flatten()   # copy
rv[0] = 99
print(X[0,0])      # may change because rv often views original

99


In [6]:
Y = np.arange(12)
Y2 = Y.reshape(3,4)     # view (contiguous)
Y3 = Y2.T.reshape(6,2)  # may need a copy due to transpose (non-contiguous)
print(Y2.base is Y)     # True (shares memory)

True


In [7]:
from numpy.lib.stride_tricks import as_strided

a = np.arange(10, dtype=np.int32)
# Create a sliding window view of width 4
window = 4
shape = (a.size - window + 1, window)
strides = (a.strides[0], a.strides[0])
w = as_strided(a, shape=shape, strides=strides)  # view only
print(w[:3])
# Always ensure the computed windows stay within the buffer!

[[0 1 2 3]
 [1 2 3 4]
 [2 3 4 5]]


In [8]:
Z = np.arange(8)
v = Z[::2]
c = Z[::2].copy()
print(v.base is Z, c.base is Z)   # True, False

True False


In [9]:
# 1) Prove that slicing returns a view by modifying a slice
a = np.arange(10)
b = a[3:8]
b[:] = -1
print(a)  # indices 3..7 should be -1

# 2) Show ravel vs flatten behavior on a transposed array
X = np.arange(12).reshape(3,4)
XT = X.T
r = XT.ravel()      # likely a copy now (non-contiguous)
f = XT.flatten()    # copy
r[0] = 999
print(X[0,0])       # check whether original changed

# 3) Compute strides of a 2D array and a strided subview
M = np.arange(24, dtype=np.int64).reshape(4,6)
S = M[::2, ::3]
print(M.strides, S.strides, S.shape)

# 4) Create a safe sliding window function using as_strided
def sliding_window(a, w):
    from numpy.lib.stride_tricks import as_strided
    shape = (a.size - w + 1, w)
    strides = (a.strides[0], a.strides[0])
    return as_strided(a, shape=shape, strides=strides)
print(sliding_window(np.arange(7), 3))

[ 0  1  2 -1 -1 -1 -1 -1  8  9]
0
(48, 8) (96, 24) (2, 2)
[[0 1 2]
 [1 2 3]
 [2 3 4]
 [3 4 5]
 [4 5 6]]
